<a href="https://colab.research.google.com/github/zjubirdfly/WebCola/blob/master/GeneratePicture.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Connect to google cloud and import data
import uuid
import pandas as pd
import numpy as np
import os
import glob
import shutil
from google.colab import auth
from google.colab import drive
project_id = 'directed-symbol-252319'
bucket_name = "chongqingtaxidata"
raw_data_bucket_name = "raw_taxi_data"
destination_path = "/content/Taxidata/"
auth.authenticate_user()
drive.mount('/content/drive')
!gcloud config set project {project_id}

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Updated property [core/project].


In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
#Copy files into system
!mkdir {destination_path}
!gsutil -m cp -r gs://{raw_data_bucket_name}/201703/25 {destination_path}


In [0]:
data_cols = ['date','time', 'label', 'tag','lat','lon','speed?','direction?','status', '?']
files = glob.glob("/content/Taxidata/25/*.TXT")

all_date_data = []
sample_data = []
for file in files:
    sample_data.append(pd.read_csv(file, sep=',',encoding='GB2312', names= data_cols, header=None,  dtype={'time': object}))

all_date_data = pd.concat(sample_data, axis=0, ignore_index=True)

In [0]:
# Generate N*N graph
N = 25

max_lat = max(all_date_data['lat'])
min_lat = min(all_date_data['lat'])
lat_range = (max_lat - min_lat) / N

max_lon = max(all_date_data['lon'])
min_lon = min(all_date_data['lon'])
lon_range = (max_lon - min_lon) / N

graph = np.zeros((N,N))

for row in all_date_data.iterrows(): 
  lat = row['lat']
  lon = row['lon']
  #lat index
  lat_index = 0;
  if lat < min_lat:
    lat_index = 0
  elif lat > max_lat:
    lat_index = N - 1
  else:
    lat_index = int((lat - min_lat) / N)

  #lon index
  lon_index = 0;
  if lon < min_lon:
    lon_index = 0
  elif lon > max_lon:
    lon_index = N - 1
  else:
    lon_index = int((lon - min_lon) / N)

  graph[lat_index, lon_index] = graph[lat_index, lon_index] + 1
